# Projeto 1 Visão Computacional 2020
## Gabriel Zezze & Pedro Luiz

Importando pacotes

In [3]:
import G6_iris_recognition
import cv2
import os
import numpy as np
import shutil

Montando o google drive localmente para ter acesso as imagens

In [4]:
RAW_IMAGES_PATH = '../assets/raw_images'
SHARPENED_IMAGES_PATH = './sharpened_images'
TEST_IMAGES_PATH = './test_images'

shutil.rmtree(SHARPENED_IMAGES_PATH, ignore_errors=True)
shutil.rmtree(TEST_IMAGES_PATH, ignore_errors=True)

if not os.path.exists(SHARPENED_IMAGES_PATH):
  os.mkdir(SHARPENED_IMAGES_PATH)

if not os.path.exists(TEST_IMAGES_PATH):
  os.mkdir(TEST_IMAGES_PATH)

Blacklist de imagens invalidas de treinamentos prévios

In [5]:
blacklist = [
    "0001/0001_006.bmp",
    "0001/0001_011.bmp",
    "0001/0001_005.bmp",
    "0001/0001_004.bmp",
    "0001/0001_012.bmp",
    "0000/0000_003.bmp",
    "0000/0000_008.bmp",
    "0000/0000_016.bmp",
    "0000/0000_000.bmp",
    "0000/0000_017.bmp",
    "0002/0002_016.bmp",
    "0002/0002_017.bmp",
    "0002/0002_013.bmp",
    "0002/0002_014.bmp",
    "0002/0002_015.bmp",
    "0002/0002_012.bmp",
    "0002/0002_003.bmp",
    "0003/0003_004.bmp",
    "0003/0003_005.bmp",
    "0003/0003_013.bmp",
    "0003/0003_017.bmp",
    "0003/0003_016.bmp",
    "0003/0003_018.bmp",
    "0003/0003_000.bmp",
    "0003/0003_011.bmp"
]
# blacklist = []

Utilities Functions

In [6]:
def equalize_img(img):
  return cv2.equalizeHist(img)

def crop_image(img, cordenadas1, cordenadas2):
  x1, y1 = cordenadas1
  x2, y2 = cordenadas2

  h, w = img.shape
  return img[y1 : h - y2][x1 : w - x2]

In [7]:
cont = 0
cont2 = 0
for folder in ["0000", "0001", "0002", "0003"]: #os.listdir(RAW_IMAGES_PATH):
  cont2 = 0
  if cont == 4:
    break
  cont += 1
  if folder == '.ipynb_checkpoints' or folder == '.DS_Store':
    continue
  # folder = "0003"
  if not os.path.exists(f'{SHARPENED_IMAGES_PATH}/{folder}'):
    os.mkdir(f'{SHARPENED_IMAGES_PATH}/{folder}')
  if not os.path.exists(f'{TEST_IMAGES_PATH}/{folder}'):
    os.mkdir(f'{TEST_IMAGES_PATH}/{folder}')
  for file in os.listdir(f'{RAW_IMAGES_PATH}/{folder}'):
    if f"{folder}/{file}" in blacklist:
      continue
    # if cont2 >= 7:
    #   continue
    # cont2 += 1

    img = cv2.cvtColor(cv2.imread(f'{RAW_IMAGES_PATH}/{folder}/{file}'), cv2.COLOR_BGR2GRAY)
    h, w = img.shape

    img_cropped = img
    y2 = img_cropped.shape[0]
    y1 = 0

    initial_crop = (20, 35)
    crop_cord = initial_crop

    while abs(y2 - y1) > img_cropped.shape[0] * 0.70 and not img_cropped.shape[0] < img.shape[1] * 0.5:

      crop_cord = (initial_crop[0], crop_cord[1] + 2)
      crop_cord2 = initial_crop
      img_cropped = crop_image(img_cropped, crop_cord, crop_cord2)

      img_equalized = equalize_img(img_cropped)
      img_blur = cv2.medianBlur(img_equalized, 25)

      img_border_detected = cv2.Canny(img_blur, 50, 100)

      pts = np.argwhere(img_border_detected > 0)
      y1,x1 = pts.min(axis=0)
      y2,x2 = pts.max(axis=0)

    blank_image = np.zeros((h, w), np.uint8)
    final_crop = img_equalized[y1 : y2 , x1 : x2]

    blank_image[ y1 : y2, x1 : x2 ] = final_crop

    final_crop = blank_image

    # circles = cv2.HoughCircles(cropped,cv2.HOUGH_GRADIENT,1,280,param1=35,param2=30,minRadius=40,maxRadius=100)
    img_equalized = cv2.cvtColor(img_equalized, cv2.COLOR_GRAY2BGR)
    rec = cv2.rectangle(img_equalized, (x1,y1), (x2,y2), (0,0,255), 3, cv2.LINE_AA)
    # out = np.where(rec == np.array([255,255,255]), img, img_blur)
    # cv2.imshow(out)
    # if circles is not None:
    #   circles = np.round(circles[0,:]).astype("int")
    #   for (x, y, r) in circles:
    #     cv2.circle(img_equalized, (x, y), r, (0, 255, 0), 4)
    if cont2 < 7:
      cv2.imwrite(f'{SHARPENED_IMAGES_PATH}/{folder}/{file}', final_crop)
    else:
      cv2.imwrite(f'{TEST_IMAGES_PATH}/{folder}/{file}', final_crop)
    cont2 += 1

In [8]:
G6_iris_recognition.iris_model_train(SHARPENED_IMAGES_PATH, "model.pickle")

directory_list ['./sharpened_images/0001', './sharpened_images/0002', './sharpened_images/0000', './sharpened_images/0003']
name 0001
path_to_image ./sharpened_images/0001/0001_013.bmp
path_to_image ./sharpened_images/0001/0001_014.bmp
path_to_image ./sharpened_images/0001/0001_002.bmp
path_to_image ./sharpened_images/0001/0001_015.bmp
path_to_image ./sharpened_images/0001/0001_003.bmp
path_to_image ./sharpened_images/0001/0001_007.bmp
path_to_image ./sharpened_images/0001/0001_000.bmp
name 0002
path_to_image ./sharpened_images/0002/0002_002.bmp
path_to_image ./sharpened_images/0002/0002_005.bmp
path_to_image ./sharpened_images/0002/0002_004.bmp
path_to_image ./sharpened_images/0002/0002_009.bmp
path_to_image ./sharpened_images/0002/0002_007.bmp
path_to_image ./sharpened_images/0002/0002_011.bmp
path_to_image ./sharpened_images/0002/0002_018.bmp
name 0000
path_to_image ./sharpened_images/0000/0000_007.bmp
path_to_image ./sharpened_images/0000/0000_009.bmp
path_to_image ./sharpened_imag

['0001', '0002', '0000', '0003']

In [ ]:
for folder in os.listdir(TEST_IMAGES_PATH):
  if folder == '.ipynb_checkpoints' or folder == '.DS_Store':
    continue
    
  for file in os.listdir(f'{TEST_IMAGES_PATH}/{folder}'):
    if f"{folder}/{file}" in blacklist:
      continue
    iris_name = G6_iris_recognition.iris_model_test("model.pickle", f'{RAW_IMAGES_PATH}/{folder}/{file}')
    print(iris_name, folder, file)
    # if iris_name != 'unmatch':
    #   print(iris_name)
    print("---------- \n")
    

0001 0001 0001_009.bmp
---------- 

0003 0001 0001_018.bmp
---------- 

0001 0001 0001_016.bmp
---------- 

0001 0001 0001_008.bmp
---------- 

0001 0001 0001_001.bmp
---------- 

0001 0001 0001_017.bmp
---------- 

0000 0001 0001_010.bmp
---------- 

0001 0001 0001_019.bmp
---------- 

0000 0002 0002_008.bmp
---------- 

0000 0002 0002_001.bmp
---------- 

0002 0002 0002_006.bmp
---------- 

0000 0002 0002_010.bmp
---------- 

0001 0002 0002_019.bmp
---------- 

0002 0002 0002_000.bmp
---------- 

0000 0000 0000_005.bmp
---------- 

0000 0000 0000_002.bmp
---------- 

0001 0000 0000_013.bmp
---------- 

0000 0000 0000_014.bmp
---------- 

0000 0000 0000_004.bmp
---------- 

